# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [152]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [154]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
df = pd.get_dummies(df, columns=["home_ownership", "verification_status","hardship_flag","debt_settlement_flag","issue_d","pymnt_plan","application_type","initial_list_status","next_pymnt_d"])

# Preview the data
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,1,1,0,0,1,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0,0,1,1,1,0,0,1,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0,0,1,1,1,0,0,1,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,1,1,0,0,1,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,0,1,1,1,0,0,1,0,1


## Split the Data into Training and Testing

In [155]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y =df["loan_status"]
y.head()

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [156]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.371696,0.451066,0.177238,1.0,0.860340,0.139660,0.123879,0.876121,0.383161,0.616839
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.483261,0.497603,0.381873,0.0,0.346637,0.346637,0.329446,0.329446,0.486161,0.486161
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,0.000000,1.0,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [157]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [158]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,
                                                   y,
                                                   random_state=1,
                                                   stratify=y)
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [159]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [160]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [161]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [162]:
# Create a random forest classifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=78)
# Resample the training data with the BalancedRandomForestClassifier
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

In [163]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, predictions)
print(acc_score)

0.9019471083987213


In [164]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

matrix = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    matrix, index=["Actual High-Risk", "Actual Low-Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])
cm_df

,Predicted High_Risk,Predicted Low_Risk
Actual High-Risk,59,28
Actual Low-Risk,1659,15459


In [165]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.90      0.07      0.78      0.60        87
   low_risk       1.00      0.90      0.68      0.95      0.78      0.63     17118

avg / total       0.99      0.90      0.68      0.94      0.78      0.63     17205



In [166]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.06949794660629233, 'last_pymnt_amnt'),
 (0.06838768711754954, 'total_rec_prncp'),
 (0.06187727039749519, 'total_rec_int'),
 (0.06140972349105115, 'total_pymnt_inv'),
 (0.050525395963678756, 'total_pymnt'),
 (0.027666832727408705, 'int_rate'),
 (0.018739369529793704, 'issue_d_Jan-2019'),
 (0.017598517096615973, 'max_bal_bc'),
 (0.01747812829661088, 'installment'),
 (0.017316206673222492, 'mths_since_recent_inq'),
 (0.016880829529775085, 'annual_inc'),
 (0.01685849647540544, 'dti'),
 (0.01680383866083915, 'il_util'),
 (0.016283192036219207, 'out_prncp'),
 (0.016018759178375898, 'mths_since_rcnt_il'),
 (0.01591511414338516, 'tot_hi_cred_lim'),
 (0.01481863364673417, 'revol_bal'),
 (0.014567026518848319, 'mo_sin_old_rev_tl_op'),
 (0.014339342937031707, 'bc_util'),
 (0.014210161457150244, 'total_rev_hi_lim'),
 (0.01399891735899933, 'total_bc_limit'),
 (0.013875519140365633, 'out_prncp_inv'),
 (0.013855815551959121, 'mo_sin_old_il_acct'),
 (0.013450759539593782, 'avg_cur_bal'),
 (0.01342

### Easy Ensemble Classifier

In [167]:
# Train the Classifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=61)

eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=61)

In [168]:
# Calculated the balanced accuracy score
e_acc_score = balanced_accuracy_score(y_test, predictions)
print(f"Balanced Accuracy Score : {e_acc_score}")

Balanced Accuracy Score : 0.7906226960126665


In [169]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

In [170]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.90      0.07      0.78      0.60        87
   low_risk       1.00      0.90      0.68      0.95      0.78      0.63     17118

avg / total       0.99      0.90      0.68      0.94      0.78      0.63     17205



### Final Questions

1. Which model had the best balanced accuracy score?

In [171]:
print(f"Balanced Random Forest = {acc_score} ")
print(f"Easy Ensembly = {e_acc_score}")


Balanced Random Forest = 0.9019471083987213 
Easy Ensembly = 0.7906226960126665



2. Which model had the best recall score?

In [172]:
print(f"Balanced Random Forest = 0.90 ")
print(f"Easy Ensembly = 0.95")

Balanced Random Forest = 0.90 
Easy Ensembly = 0.95


3. Which model had the best geometric mean score?

In [173]:
print(f"Balanced Random Forest = 0.78 ")
print(f"Easy Ensembly = 0.78")

Balanced Random Forest = 0.78 
Easy Ensembly = 0.78


4. What are the top three features?

In [174]:
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)[:3]


[(0.06949794660629233, 'last_pymnt_amnt'),
 (0.06838768711754954, 'total_rec_prncp'),
 (0.06187727039749519, 'total_rec_int')]